In [17]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from glob import glob

import keras

In [18]:
def load_tcr_dataset_from_dir(data_dir, batch_size=100, shuffle=True):
    # Collect Files
    df_rep = pd.DataFrame(glob(os.path.join(data_dir, '*/*.tsv')), columns=['filepath'])
    df_rep[['HLA', 'Antigen']] = df_rep['filepath'].str.extract(r'/([^/-]+)-([^/-]+)/[^/]+$')
    
    # Load TSV into df
    df_tcr = []
    for idx in df_rep.index:
        df = pd.read_csv(df_rep.loc[idx, 'filepath'], sep='\t')
        df['index'] = idx
        df[['HLA', 'Antigen']] = df_rep.loc[idx, ['HLA', 'Antigen']].values
        df_tcr.append(df)
    df_tcr = pd.concat(df_tcr)
    df_tcr['Antigen'] = df_tcr['Antigen'].astype('category')

    # Embedding
    idx = ~df_tcr[['aminoAcid', 'Antigen']].isna().any(axis=1)
    X = df_tcr.loc[idx, 'aminoAcid'].values
    y = df_tcr.loc[idx, 'Antigen'].cat.codes.values
    
    # Check if sequences exist
    if len(X) == 0:
        raise ValueError("No valid amino acid sequences found in the data")
    
    label_map = dict(enumerate(df_tcr.loc[idx, 'Antigen'].cat.categories))

    max_length = max(map(len, X))
    vocab = sorted(set(''.join(X)))
    aa_dict = {aa: i + 1 for i, aa in enumerate(vocab)}

    # Padding
    X_encoded = np.zeros((len(X), max_length), dtype=np.int32)
    for i, seq in enumerate(X):
        for j, aa in enumerate(seq[:max_length]):
            X_encoded[i, j] = aa_dict.get(aa, 0)

    # Create TensorFlow dataset
    ds = tf.data.Dataset.from_tensor_slices((X_encoded, y))
    ds = ds.map(lambda x, y: (x, tf.one_hot(y, len(label_map))))
    if shuffle:
        ds = ds.shuffle(len(X_encoded))
    ds = ds.batch(batch_size, drop_remainder = True).prefetch(tf.data.AUTOTUNE)

    return ds, aa_dict, label_map

In [19]:
data_dir = '/projects/deeptcr/DeepTCR/Data/Human_Antigens'

In [20]:
tfds, aa_dict, y= load_tcr_dataset_from_dir(data_dir)

In [21]:
tfds

<_PrefetchDataset element_spec=(TensorSpec(shape=(100, 23), dtype=tf.int32, name=None), TensorSpec(shape=(100, 7), dtype=tf.float32, name=None))>

In [22]:
aa_dict

{'A': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'K': 9,
 'L': 10,
 'M': 11,
 'N': 12,
 'P': 13,
 'Q': 14,
 'R': 15,
 'S': 16,
 'T': 17,
 'V': 18,
 'W': 19,
 'Y': 20}

In [23]:
# CDR3 AA encoding
tensors = []
tensors.append(keras.layers.Input(shape=(23, ),  dtype=tf.uint32))
tensors.append(keras.layers.Embedding(input_dim=len(aa_dict) + 1, output_dim=64, mask_zero=True)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=64, kernel_size=9, strides=1, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=128, kernel_size=7, strides=2, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, activation=None)(tensors[-1])[:, 0, :])
encoder = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='encoder')

In [24]:
# VDJ usage encoding
tensors = []
tensors.append(keras.layers.Input(shape=(23, ),  dtype=tf.uint32))
tensors.append(keras.layers.Embedding(input_dim=len(aa_dict) + 1, output_dim=64, mask_zero=True)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=64, kernel_size=9, strides=1, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=128, kernel_size=7, strides=2, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, activation=None)(tensors[-1])[:, 0, :])
encoder = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='encoder')

In [25]:
tensors = []
tensors.append(keras.layers.Input(shape=(encoder.output_shape[-1], ), dtype=tf.float32))
tensors.append(keras.layers.Dropout(rate=0.1)(tensors[-1]))
tensors.append(keras.layers.Dense(units=128, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Dropout(rate=0.05)(tensors[-1]))
tensors.append(keras.layers.Dense(units=64, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Dense(units=len(y), activation=keras.activations.sigmoid)(tensors[-1]))
classifier = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='classifer')

In [26]:
tensors = []
tensors.append(keras.layers.Input(shape=encoder.input_shape[1:],  dtype=tf.uint32))
tensors.append(encoder(tensors[-1]))
tensors.append(classifier(tensors[-1]))
model = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='full_model')

In [27]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.BinaryCrossentropy(from_logits=False)
)

In [28]:
model.fit(tfds, epochs=100)

Epoch 1/100


2025-06-18 19:10:43.050967: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2025-06-18 19:10:44.885052: I external/local_xla/xla/service/service.cc:168] XLA service 0x154f58ba03f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-06-18 19:10:44.885093: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-06-18 19:10:44.890399: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1750288244.992007  532635 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20/20 [==============================] - 5s 4ms/step - loss: 0.4786
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3557
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3512
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.3493
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3430
Epoch 6/100
20/20 [==============================] - 0s 11ms/step - loss: 0.3314
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3176
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3041
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2924
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2750
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2554
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2277
Epoch 13/100
20/20 [============================

In [29]:
b = next(iter(tfds))

In [30]:
b[0].shape, b[1].shape

(TensorShape([100, 23]), TensorShape([100, 7]))